<a href="https://colab.research.google.com/github/ssonone/FutureTechTrajectories/blob/main/WoSDB_Table_Download_general.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Modules
import os
import pandas as pd
from sqlalchemy import create_engine
from tqdm.auto import tqdm
import pymysql
from dotenv import load_dotenv

# pymysql을 MySQLdb로 사용하기 위한 설정
pymysql.install_as_MySQLdb()


# 환경변수 파일 로드
load_dotenv()

In [ ]:
def get_database_engine():
    # 환경변수에서 데이터베이스 연결 정보 가져오기
    user = os.getenv('DB_USER')
    pw = os.getenv('DB_PASSWORD')
    ip = os.getenv('DB_IP')
    port = os.getenv('DB_PORT')
    db = os.getenv('DB_NAME')
    return create_engine(f"mysql://{user}:{pw}@{ip}:{port}/{db}")

In [ ]:
def fetch_data(engine, year, table_name):
    query = f"""
    SELECT B.* FROM `{table_name}` B
    JOIN `pub_info` A ON A.uid = B.uid
    WHERE A.pubyear = %s;
    """
    # pub_info ; pubyear를 포함하는 publication infomation 테이블
    return pd.read_sql_query(query, engine, params=[(year,)])

In [ ]:
def main():
    engine = get_database_engine()
    dataframes = []

    # 데이터베이스 이름 환경변수에서 불러오기
    db_name = os.getenv('DB_NAME')

    # 사용자로부터 처리할 테이블 이름 입력 받기
    table_name = input("Enter the table name to process (e.g., title, abstract, keyword): ")

    for year in tqdm(range(2006, 2024), desc=f"Processing {table_name}"):
        df = fetch_data(engine, year, table_name)
        dataframes.append(df)

    combined_df = pd.concat(dataframes, ignore_index=True)
    combined_df.to_pickle(f'combined_df_{db_name}_{table_name}.pkl')
    engine.dispose()

In [ ]:

if __name__ == "__main__":
    main()
